## **ChatBot**

#### **Importing Libraries**

In [31]:
import random
import json
import pickle
import numpy as np

import pyttsx3

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import load_model

import nbimporter
import Audio_Cleaning

from deep_translator import GoogleTranslator

In [32]:
lemmatizer = WordNetLemmatizer()

# https://www.kaggle.com/code/aminianam/simple-chatbot/input
intents = json.loads(open('Intent English.json').read())

words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbotmodel.h5')

#### **Function for Cleaning Up the Sentences**

In [33]:
def cleaning_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

#### **Function for Bag-of-Words**

In [34]:
def bag_of_words(sentence):
    sentence_words = cleaning_up_sentence(sentence)
    bag = [0] * len(words)

    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    
    return np.array(bag)

#### **Function for Predicting**

In [35]:
def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    result = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    result.sort(key = lambda x: x[1], reverse = True) # highest probability first
    return_list = []

    for r in result:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    
    return return_list

#### **Getting Response Function**

In [36]:
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']

    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    
    return result

In [39]:
import import_ipynb
import time
# %run Users/cb28v/OneDrive - Alberta Energy Regulator/Documents/Project/Capstone/Speech Language Detector Testing.ipynb

languages = ['en', 'fr', 'es']
print('GO! Bot is running!')

while True:
    # %run ./Testing.ipynb
    # %run ./Speech_Lang_Detector.ipynb
    %run ./Audio_Cleaning.ipynb

    possible_langs = Audio_Cleaning.lang_detect(text_en, text_fr, text_es)
    lang_list, prob_list = Audio_Cleaning.lang_prob(possible_langs)
    lang_ISO = Audio_Cleaning.ISO_639(lang_list, prob_list)

    if languages.index(lang_ISO) == 0:
        message = text_en
        print('You: {}'.format(message))

        if message.lower() == 'I would like to exist please':
            break

        time.sleep(1)

        ints = predict_class(message)
        res = get_response(ints, intents)
        print('Bot: {}\n'.format(res))

        engine = pyttsx3.init()
        voices = engine.getProperty('voices')
        engine.setProperty('voice', voices[1].id)
        engine.say(res)
        engine.runAndWait()

    if languages.index(lang_ISO) == 1:
        message = text_fr
        print('You: {}'.format(message))
        
        time.sleep(1)

        translator_fr2en = GoogleTranslator(source = 'auto', target = 'en')
        fr2en = translator_fr2en.translate(message)
        print('Translated FR 2 EN: ', translator_fr2en.translate(message))

        if fr2en.lower() == 'I would like to exist please':
            break

        ints = predict_class(fr2en)
        res = get_response(ints, intents)
        translator_en2fr = GoogleTranslator(source = 'auto', target = 'fr')
        res_en2fr = translator_en2fr.translate(res)
        print('Bot: {}\n'.format(res_en2fr))

        engine = pyttsx3.init()
        voices = engine.getProperty('voices')
        engine.setProperty('voice', voices[3].id)
        engine.say(res_en2fr)
        engine.runAndWait()

    if languages.index(lang_ISO) == 2:
        message = text_es
        print('You: {}'.format(message))
        
        time.sleep(1)

        translator_es2en = GoogleTranslator(source = 'auto', target = 'en')
        es2en = translator_es2en.translate(message)
        print('Translated ES 2 EN: ', translator_es2en.translate(message))

        if es2en.lower() == 'I would like to exist please':
            break

        ints = predict_class(es2en)
        res = get_response(ints, intents)
        translator_en2es = GoogleTranslator(source = 'auto', target = 'es')
        res_en2es = translator_en2fr.translate(res)
        print('Bot: {}\n'.format(res_en2es))

        engine = pyttsx3.init()
        voices = engine.getProperty('voices')
        engine.setProperty('voice', voices[2].id)
        engine.say(res_en2es)
        engine.runAndWait()
        
    # message = text_en
    # message = input("You: ")
    # %store -r text_en
    # message = text_en
    # print('You: {}'.format(message))
    # time.sleep(1)

    # if text_en.lower() == 'quit':
    #     break
    
    # ints = predict_class(message)
    # res = get_response(ints, intents)
    # print('Bot: {}'.format(res))
    # print()

    # engine = pyttsx3.init()
    # engine.say(res)
    # engine.runAndWait()

    # break

    time.sleep(2)

GO! Bot is running!
Converting audio transcripts into text ...
Stored 'text_en' (str)

Sorry.. run again...
[['en:0.9999970117862897'], ['pt:0.9999919508244953'], ['es:0.9999953920958244']]
You: hello how's it going
1/1 [==============================] - 0s 23ms/step
Bot: Welcome to the border security checkpoint. How can I assist you?

Converting audio transcripts into text ...
Stored 'text_en' (str)

Converting audio transcripts into French text ...
Stored 'text_fr' (str)

Converting audio transcripts into Spanish text ...
Stored 'text_es' (str)

[['hr:0.4516030173120512', 'fr:0.4285699715344995', 'sl:0.11982562624395718'], ['fr:0.9999972773089126'], ['ca:0.5943319638441391', 'es:0.40566651919069785']]
Langauge ISO 639-2:  fr
You: bonjour comment ça va
Translated FR 2 EN:  Hello how are you
1/1 [==============================] - 0s 20ms/step
Bot: Merci d'avoir fourni vos documents. Laissez-moi les vérifier.

Converting audio transcripts into text ...
Stored 'text_en' (str)

Convertin

KeyboardInterrupt: 

In [ ]:
# %run ./Speech_Lang_Detector.ipynb
%run ./Speech_Lang_Detector.ipynb

In [ ]:
x = Speech_Lang_Detector.audio()
x

In [ ]:
# !pip install nbimporter
import nbimporter
import Audio_Cleaning

In [ ]:
%run ./Audio_Cleaning.ipynb

In [ ]:
text_en

In [ ]:
text_fr

In [ ]:
text_es

In [ ]:
x = Audio_Cleaning.lang_detect(text_en, text_fr, text_es)
x

In [ ]:
a, b = Audio_Cleaning.lang_prob(x)
print(a)
print(b)

In [ ]:
y = Audio_Cleaning.ISO_639(a, b)
y

In [ ]:
languages = ['en', 'fr', 'es']

languages.index(y)

In [ ]:
x = {'fr': 2, 'en': 1, 'es': 1}
max(x, key = x.get)

In [ ]:
import pyttsx3
engine = pyttsx3.init()
voices = engine.getProperty('voices')
for voice in voices:
    print(voice, voice.id)

<Voice id=HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_DAVID_11.0
          name=Microsoft David Desktop - English (United States)
          languages=[]
          gender=None
          age=None> HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_DAVID_11.0
<Voice id=HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_ZIRA_11.0
          name=Microsoft Zira Desktop - English (United States)
          languages=[]
          gender=None
          age=None> HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_ZIRA_11.0
<Voice id=HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_ES-MX_SABINA_11.0
          name=Microsoft Sabina Desktop - Spanish (Mexico)
          languages=[]
          gender=None
          age=None> HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_ES-MX_SABINA_11.0
<Voice id=HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_FR-FR_HORTENS

In [ ]:
voices[3].id

'HKEY_LOCAL_MACHINE\\SOFTWARE\\Microsoft\\Speech\\Voices\\Tokens\\TTS_MS_FR-FR_HORTENSE_11.0'

In [ ]:
engine.setProperty('voice', voices[3].id)